Last week I had the priveledge to attend the MLSS, a Machine Learning conference/summer school where some of the most well-known ML researchers gave 4.5 hour presentations. At one point David Blai and Frank Wood spoke back and forth about the idea of using a generative model to augment a dataset that would be used to train a discrimanitive model. Blai was sceptical since the generative model would have been trained on the same information as the discrimanitive and thus wouldnt create any new information, but Wood countered that Blai was missing one detail. The generative model would have _structure_ built into it. Any prior knowledge that we have about the dataset could be programed into the generative machinery, and that prior knowledge could theoretically bring something new to the table when training the discriminative model. 

In this notebook I attempt to measure this information gain with our favorite discriminative machine learning problem, MNIST.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

from datetime import datetime

from utils import *

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
tf.reset_default_graph()

In [8]:
X = tf.placeholder(tf.float32, shape=[None, 784])
X_reshape = tf.reshape(X, [-1, 28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# def classifier(img):
#     h1 = conv2d_with_pooling(img, 1, 16, 'conv_1') # 28x28x1 -> 14x14x16
#     h2 = conv2d_with_pooling(h1, 16, 32, 'conv_2')# 14x14x16 -> 7x7x32
#     h2_flat = tf.reshape(h2,[-1, 7*7*32])
#     out = dense(h2_flat, 7*7*32, 10, "w_mean")
#     return out

def classifier(img):
    h1 = lrelu(conv2d(img, 1, 16, 'conv_1')) # 28x28x1 -> 14x14x16
    h2 = lrelu(conv2d(h1, 16, 32, 'conv_2'))# 14x14x16 -> 7x7x32
    h2_flat = tf.reshape(h2,[-1, 7*7*32])
    out = dense(h2_flat, 7*7*32, 10, "w_mean")
    return out

logits = classifier(X_reshape)

loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=logits))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
#no pooling
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, 
                                  feed_dict={X: batch[0], y_: batch[1]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, 
             feed_dict={X: batch[0], y_: batch[1]})

print('test accuracy %g' % sess.run(accuracy, 
                                    feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.88
step 300, training accuracy 0.94
step 400, training accuracy 0.9
step 500, training accuracy 0.94
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 1
test accuracy 0.9669


In [6]:

for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, 
                                  feed_dict={X: batch[0], y_: batch[1]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, 
             feed_dict={X: batch[0], y_: batch[1]})

print('test accuracy %g' % sess.run(accuracy, 
                                    feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


step 0, training accuracy 0.08
step 100, training accuracy 0.9
step 200, training accuracy 0.88
step 300, training accuracy 0.94
step 400, training accuracy 0.96
step 500, training accuracy 0.98
step 600, training accuracy 1
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.98
test accuracy 0.9757


In [103]:
train_data = mnist.train.next_batch(5000)
ti = train_iterator(train_data)

for i in range(2000):
    batch = ti.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, 
                                  feed_dict={X: batch[0], y_: batch[1]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, 
             feed_dict={X: batch[0], y_: batch[1]})

print('test accuracy %g' % sess.run(accuracy, 
                                    feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


step 0, training accuracy 0.08
step 100, training accuracy 0.92
step 200, training accuracy 0.94
step 300, training accuracy 0.96
step 400, training accuracy 0.94
step 500, training accuracy 0.98
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 1
step 1300, training accuracy 1
step 1400, training accuracy 0.98
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 1
test accuracy 0.966


In [108]:
train_data = mnist.train.next_batch(200)
ti = train_iterator(train_data)

for i in range(2000):
    batch = ti.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, 
                                  feed_dict={X: batch[0], y_: batch[1]})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, 
             feed_dict={X: batch[0], y_: batch[1]})

print('test accuracy %g' % sess.run(accuracy, 
                                    feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


step 0, training accuracy 0.06
step 100, training accuracy 1
step 200, training accuracy 1
step 300, training accuracy 1
step 400, training accuracy 1
step 500, training accuracy 1
step 600, training accuracy 1
step 700, training accuracy 1
step 800, training accuracy 1
step 900, training accuracy 1
step 1000, training accuracy 1
step 1100, training accuracy 1
step 1200, training accuracy 1
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 1
test accuracy 0.8506
